In [15]:
from pyiron_workflow import Workflow
from PIL import Image
from pathlib import Path
import matplotlib.pylab as plt
import numpy as np
import os
import shutil
import base64
import io
import pylnk3
import re
import subprocess
import uuid
import time
from pyironflow.pyironflow import PyironFlow

In [16]:
### sub_function_node: decide each time, overwrite or create a new folder.

## every time a new directory, not overwrite
@Workflow.wrap.as_function_node
def generate_working_directory_keep(path, directory_name):
    base = Path(path)
    base.mkdir(parents=True, exist_ok=True)

    # find the biggest x in existed "Python_marco_to_inp_x" 
    existing = [d for d in base.iterdir() if d.is_dir() and d.name.startswith(directory_name)]
    if existing:
        numbers = [int(d.name.rsplit("_", 1)[1]) for d in existing if "_" in d.name]
        next_num = max(numbers) + 1
    else:
        next_num = 1
    working_directory = base / f"{directory_name}_{next_num}"
    working_directory.mkdir()
    return working_directory


## every time overwrite the directory
@Workflow.wrap.as_function_node
def generate_working_directory_overwrite(path, directory_name):
    working_directory = Path(path) / directory_name
    print(f"[INFO] Working directory: {working_directory}")

    # 
    if not hasattr(generate_working_directory_overwrite, "_already_initialized"):
        if working_directory.exists():
            print("[INFO] (First run) Directory exists — deleting it and recreating.")
            shutil.rmtree(working_directory)  
        working_directory.mkdir(parents=True, exist_ok=True)
        generate_working_directory_overwrite._already_initialized = True
        print("[INFO] (First run) Directory ready.")
    else:
        # 
        if not working_directory.exists():
            print("[INFO] (Later run) Directory missing — creating it again.")
            working_directory.mkdir(parents=True, exist_ok=True)
        else:
            print("[INFO] (Later run) Directory already exists — keep using it (no delete).")

    return working_directory

In [17]:
### Marco_node: Geometry change

@Workflow.wrap.as_function_node
def update_geometry_marco(
                          original_abaqus_script,
                          working_directory,
                          plate_thickness: float = 2.0,  # plate thickness
                          plate_length: float = 16.0, # plate length
                          electrode_height: float = 10.0, # electrode height
                          electrode_diameter: float = 20.0, # electrode diameter
                          electrode_spherical_radius: float = 100.0, # electrode spherical radius
                          Partition_position: float = 4.0,  # Partition position (Macro 1)
                          Partition_position_1: float = 1.0, # Partition position 1 (Macro 3)
                          Partition_position_2: float = 1.0 # Partition position 2 (Macro 3)
                         ):

    replacements = {
                    "dB": plate_thickness,
                    "hB": plate_length,
                    "hE": electrode_height,
                    "dE": electrode_diameter,
                    "R": electrode_spherical_radius,
                    "pP": Partition_position,
                    "pP1": Partition_position_1,
                    "pP2": Partition_position_2
                   }

    with open(original_abaqus_script, "r", encoding="utf-8") as f:
        lines = f.readlines()

    inside_macro_1_or_3 = False
    updated_lines = []

    for i, line in enumerate(lines):
        if re.match(r"\s*def\s+Macro_1_Geometrie\s*\(", line) or re.match(r"\s*def\s+Macro_3_", line):
            inside_macro_1or3 = True
        elif inside_macro_1_or_3 and re.match(r"\s*def\s+\w+", line):
            inside_macro_1_or_3 = False
        if inside_macro_1_or_3:
            stripped = line.strip()
            matched = False
            for var, val in replacements.items():
                if stripped.startswith(f"{var} =") or stripped.startswith(f"{var}="):
                    indent = line[:line.find(var)]
                    new_line = f"{indent}{var} = {val}\n"
                    updated_lines.append(new_line)
                    matched = True
                    break
                if not matched:
                    updated_lines.append(line)
        else:
            updated_lines.append(line)

    base_name = os.path.splitext(original_abaqus_script)[0]
    base_name = base_name + '_Geo_modified' 
    new_py_path = os.path.join(working_directory, base_name + ".py")
    new_txt_path = os.path.join(working_directory, base_name + ".txt")
    
    # new abaqus script
    with open(new_py_path, "w", encoding="utf-8") as f:
        f.writelines(updated_lines)

    # meanwhile a .txt, include the changed parameters
    with open(new_txt_path, "w", encoding="utf-8") as f:
        for var, val in replacements.items():
            f.write(f"{var} = {val}\n")
            
    return new_py_path, new_txt_path

In [18]:
### Marco_node: Abaqus marco script to inp 

@Workflow.wrap.as_function_node
def write_abaqus_input(script_path, working_directory):
    input_script = os.path.basename(script_path)   # Script name only (no path)
    local_input_script = os.path.join(working_directory, input_script)   # Spell out the target path in the working directory
    shutil.copy(script_path, local_input_script)
    return local_input_script


@Workflow.wrap.as_function_node
def run_in_abaqus(executable, script_path, working_directory):
    ## Construct and run Abaqus on the command line
    cmd = f'"{executable}" cae noGUI={script_path}'
    subprocess.check_call(cmd, cwd=working_directory, shell=True)   ## Run the command in the working directory
    
    # print(working_directory)
    files = os.listdir(working_directory)
    # print(files)
    
    while True:
        files = os.listdir(working_directory) # "C:/Local_Dong/Projekt/AnAttAl/CAD/AnAttAl_CAD_OnlyWorkflow_demo/Python_marco_to_inp")
        # print(files)
        cae_files = [f for f in files if f.lower().endswith('.cae')]
        inp_files = [f for f in files if f.lower().endswith('.inp')]
        if cae_files and inp_files:
            print(f"Finish, all files： {cae_files}, {inp_files}\n")
            break
    # if time.time() - start_time > timeout:
    #     raise TimeoutError("check the code")    
    time.sleep(2)
        
    cae_path = os.path.join(working_directory, cae_files[0])
    inp_path = os.path.join(working_directory, inp_files[0]) 
    
    return {"inp_path":inp_path,
            "cae_path":cae_path}

# @Workflow.wrap.as_function_node
# def collect_output(working_directory):
#     print(working_directory)
#     # start_time = time.time()
#     # timeout = 30
#     files = os.listdir(working_directory)
#     print(files)
#     while True:
#         files = os.listdir(working_directory) # "C:/Local_Dong/Projekt/AnAttAl/CAD/AnAttAl_CAD_OnlyWorkflow_demo/Python_marco_to_inp")
#         print(files)
#         cae_files = [f for f in files if f.lower().endswith('.cae')]
#         inp_files = [f for f in files if f.lower().endswith('.inp')]
#         if cae_files and inp_files:
#             print(f"Finish, all files： {cae_files}, {inp_files}\n")
#             break
#     # if time.time() - start_time > timeout:
#     #     raise TimeoutError("check the code")    
#     time.sleep(2)
        
#     cae_path = os.path.join(working_directory, cae_files[0])
#     inp_path = os.path.join(working_directory, inp_files[0]) 
    
#     return {"cae_path":cae_path, 
#             "inp_path":inp_path}
    ## Since it is a job object, saved in HDF5, it can be called directly without return


# @Workflow.wrap.as_function_node
# def generate_working_directory(path):
#     working_directory = Path(path) / "Python_marco_to_inp"
#     print(f"[INFO] Working directory: {working_directory}")

#     return working_directory


### Run Marco_node
@Workflow.wrap.as_macro_node("inp_path", "cae_path")
def export_inp_cae(macro, script_path, base_path, directory_name, executable=r"C:\SIMULIA\Commands\abq2018.bat"):
    # path = macro.as_path() # This gives you the path based on the workflow name
    path = base_path
    macro.work_dir = generate_working_directory_overwrite(path, directory_name)
    # macro.working_directory = generate_working_directory_keep(path)
    macro.scripts = write_abaqus_input(script_path=script_path, 
                                       working_directory=macro.work_dir)
    macro.inp_output = run_in_abaqus(executable=executable, 
                                     script_path=macro.scripts, 
                                     working_directory=macro.work_dir)
    # macro.check = collect_output(working_directory=macro.work_dir)

    # This is in principle not required, but it makes sure that the nodes are executed in the given order
    # macro.work_dir >> macro.scripts >> macro.inp_output >> macro.
    
    return (macro.inp_output["inp_path"],
            macro.inp_output["cae_path"])

In [19]:
### Marco_node: processing inp 

@Workflow.wrap.as_function_node
def write_inp_input(script_path, inp_path, working_directory):
        
    ## Extract the file name and copy it to the current project directory.
    # copy python script
    input_script = os.path.basename(script_path)
    local_script_path = os.path.join(working_directory, input_script)
    shutil.copy(script_path, local_script_path)

    # copy .inp file
    input_path = os.path.basename(inp_path)
    local_inp_path = os.path.join(working_directory, input_path)
    shutil.copy(inp_path, local_inp_path)

    return {"script_path":local_script_path, 
            "inp_path":local_inp_path}


@Workflow.wrap.as_function_node
def run_in_python(executable, script_path, inp_path, working_directory):
    ## Constructor commands: python script Input file
    cmd = [str(executable), script_path, inp_path]
    subprocess.check_call(cmd, cwd=working_directory)

    all_txt_files = [f for f in os.listdir(working_directory) if f.lower().endswith(".txt")]
    all_txt_files_path = [os.path.abspath(os.path.join(working_directory, f)) for f in os.listdir(working_directory) if f.endswith(".txt")]
    print(f"Finish, all files：{all_txt_files}\n")
    
    return {"txt_paths": all_txt_files_path} 


### Run Marco_node
@Workflow.wrap.as_macro_node("txt_paths")
def processing_inp(macro, script_path, inp_path, base_path, directory_name, executable=r"python"):
    # path = macro.as_path() # This gives you the path based on the workflow name
    path = base_path
    macro.work_dir = generate_working_directory_overwrite(path, directory_name)
    # macro.working_directory = generate_working_directory_keep(path)
    macro.scripts = write_inp_input(script_path=script_path, 
                                    inp_path=inp_path, 
                                    working_directory=macro.work_dir)
    macro.processing_inp = run_in_python(executable=executable, 
                                         script_path=macro.scripts['script_path'], 
                                         inp_path=macro.scripts['inp_path'],
                                         working_directory=macro.work_dir)
    # macro.check = collect_output(working_directory=macro.work_dir)

    return macro.processing_inp["txt_paths"]

In [20]:
### Marco_node: in the furture, maybe we can read FAMOS data automated

@Workflow.wrap.as_function_node
def FAMOS_input_txt(working_directory, input_txt_paths_von_FAMOS=[]):

    # Copy all input txt files into working directory
    local_txt_files = []
    for txt_path in input_txt_paths_von_FAMOS:
        txt_name = os.path.basename(txt_path)
        dst_path = os.path.join(working_directory, txt_name)
        txt_path = os.path.abspath(txt_path)
        shutil.copy(txt_path, dst_path)
        local_txt_files.append(dst_path)
        
    # print(local_txt_files)
    
    return {"input_txt_set": local_txt_files}

### Run Marco_node
@Workflow.wrap.as_macro_node("FAMOS_txt")
def FAMOS_Output(macro, base_path, directory_name, input_txt_paths_von_FAMOS=[]):
    path = base_path
    macro.work_dir = generate_working_directory_overwrite(path, directory_name)   
    macro.FAMOS_input_txt = FAMOS_input_txt(working_directory=macro.work_dir,
                                           input_txt_paths_von_FAMOS=input_txt_paths_von_FAMOS)
    
    return macro.FAMOS_input_txt["input_txt_set"]

In [21]:
### Marco_node: activate fortran compiler und get simulation.exe

@Workflow.wrap.as_function_node
def write_compiler_input(compiler_path, fortran_file_path, working_directory):
    compiler_path = os.path.abspath(compiler_path)
    fortran_file_path = os.path.abspath(fortran_file_path)

    input_fortran_file = os.path.basename(fortran_file_path)   # Script name only (no path)
    local_fortran_file_path = os.path.join(working_directory, input_fortran_file)   # Spell out the target path in the working directory
    shutil.copy(fortran_file_path, local_fortran_file_path)

    return {"compiler_path":compiler_path, 
            "fortran_path":local_fortran_file_path}
    
@Workflow.wrap.as_function_node
def run_fortran_in_compiler(compiler_path, local_fortran_file_path, working_directory):
    ## Parsing .lnk shortcuts (Windows)
    with open(compiler_path, 'rb') as f:
        lnk = pylnk3.parse(f)

    raw_path = lnk.path.strip('"')    # Executable path to which the shortcut points
    raw_args = lnk.arguments.strip()    # Shortcut parameters

    ## If .lnk points to cmd.exe, it is necessary to extract the .bat path from the parameter
    if raw_path.lower().endswith("cmd.exe"):
        match = re.search(r'"(?P<bat>C:.*?\.bat)"\s*(?P<args>[^"]*)', raw_args, re.IGNORECASE)
        if not match:
            raise ValueError(f"Unable to extract .bat file path from .lnk arguments with contents：{raw_args}")
        env_bat_path = match.group("bat")
        extra_args = match.group("args")
    else:
        env_bat_path = raw_path
        extra_args = raw_args

    ## Constructing the ifort compilation command
    # exe_name = self.output_exe or os.path.splitext(os.path.basename(self.local_fortran))[0] + ".exe"
    # exe_name = os.path.normpath(exe_name)
    fortran_path = os.path.normpath(local_fortran_file_path)
    compile_cmd = f'ifort "{fortran_path}"'

    setup_cmd = f'call "{env_bat_path}" {extra_args} && {compile_cmd}'

    # print(setup_cmd)

    try:
        result = subprocess.run(
            f'cmd /c "{setup_cmd}"',
            cwd=working_directory,
            shell=True,
            check=True,
            text=True,
            capture_output=True
        )
        print("compiler output:\n", result.stdout)
        if result.stderr:
            print("warning:\n", result.stderr)
    except subprocess.CalledProcessError as e:
        print("error")
        print("stdout:\n", e.stdout)
        print("stderr:\n", e.stderr)
        raise

    exe_files = [f for f in os.listdir(working_directory) if f.lower().endswith(".exe")]
    if not exe_files:
        raise FileNotFoundError(".exe file not found in the working directory")

    exe_filename = exe_files[0]
    exe_path = os.path.abspath(os.path.join(working_directory, exe_filename))
    
    print(f"Finish, generated file: ['{exe_filename}']\n")
    
    return {"fortran_exe_filename": exe_filename,
            "exe_path": exe_path}


### Run Marco_node
@Workflow.wrap.as_macro_node("fortran_exe_filename", "exe_path")
def fortran_to_exe(macro, compiler_path, fortran_file_path, base_path, directory_name):
    # path = macro.as_path() # This gives you the path based on the workflow name
    path = base_path
    macro.work_dir = generate_working_directory_overwrite(path, directory_name)
    # macro.working_directory = generate_working_directory_keep(path)
    macro.fortran_file = write_compiler_input(compiler_path=compiler_path, 
                                              fortran_file_path=fortran_file_path, 
                                              working_directory=macro.work_dir)
    macro.fortran_to_exe = run_fortran_in_compiler(compiler_path=macro.fortran_file["compiler_path"], 
                                                   local_fortran_file_path=macro.fortran_file["fortran_path"],
                                                   working_directory=macro.work_dir)

    return (macro.fortran_to_exe["fortran_exe_filename"],
            macro.fortran_to_exe["exe_path"])

In [33]:
### Marco_node: running simulation.exe in compiler

@Workflow.wrap.as_function_node
def write_running_exe_input(compiler_path, exe_path, working_directory, input_txt_paths_von_FAMOS=[], input_txt_paths_von_inp=[]):
    compiler_path = os.path.abspath(compiler_path)
    exe_path = os.path.abspath(exe_path)
    
    # Copy .exe to working directory
    exe_filename = os.path.basename(exe_path)   # Script name only (no path)
    local_exe_path = os.path.join(working_directory, exe_filename)   # Spell out the target path in the working directory
    shutil.copy(exe_path, local_exe_path)

    # Copy all input txt files into working directory
    local_txt_files = []
    for txt_path in input_txt_paths_von_FAMOS + input_txt_paths_von_inp:
        txt_name = os.path.basename(txt_path)
        dst_path = os.path.join(working_directory, txt_name)
        txt_path = os.path.abspath(txt_path)
        shutil.copy(txt_path, dst_path)
        local_txt_files.append(dst_path)
        
    # print(local_txt_files)
    
    return {"compiler_path": compiler_path,
            "exe_filename": exe_filename,
            "input_txt_set": local_txt_files}


@Workflow.wrap.as_function_node
def run_exe(compiler_path, exe_filename, working_directory):
    with open(compiler_path, 'rb') as f:
        lnk = pylnk3.parse(f)

    raw_path = lnk.path.strip('"')
    raw_args = lnk.arguments.strip()

    if raw_path.lower().endswith("cmd.exe"):
        match = re.search(r'"(?P<bat>C:.*?\.bat)"\s*(?P<args>[^"]*)', raw_args, re.IGNORECASE)
        if not match:
            raise ValueError(f"Unable to extract .bat file path from .lnk arguments with contents：{raw_args}")
        env_bat_path = match.group("bat")
        extra_args = match.group("args").strip('"')
    else:
        env_bat_path = raw_path
        extra_args = raw_args
        
    ## First call the compiler environment variable
    ## Then cd to the working directory
    ## Finally, execute the .exe file
    run_cmd = f'call "{env_bat_path}" {extra_args} && cd /d "{working_directory}" && "{exe_filename}"'

    # print(f"\n{run_cmd}\n")

    # try:
    #     result = subprocess.run(
    #         f'cmd /c "{run_cmd}"',
    #         cwd=working_directory,
    #         shell=True,
    #         check=True,
    #         capture_output=True,
    #         text=True
    #     )
    #     print(result.stdout)
    #     print(result.stderr)
        
    # except subprocess.CalledProcessError as e:
    #     print("error：")
    #     print("STDOUT:\n", e.stdout)
    #     print("STDERR:\n", e.stderr)
    #     raise
    
    result = subprocess.run(
                            f'cmd /c "{run_cmd}"',
                            cwd=working_directory,
                            shell=True,
                            # check=True,
                            capture_output=True,
                            text=True
                            )
    
    print(result.stdout)
    print(result.stderr)
  
    if result.returncode != 0:
        stderr_lower = result.stderr.lower()
        if "end-of-file during read" in stderr_lower and "severe (24)" in stderr_lower:
            print("Warning: Fortran EOF read error (severe 24) detected.")
            print("Assuming simulation finished successfully, continuing...\n")
        else:
            print("Simulation failed with unknown error:")
            print("STDOUT:\n", result.stdout)
            print("STDERR:\n", result.stderr)
            raise subprocess.CalledProcessError(returncode=result.returncode,
                                                cmd=run_cmd,
                                                output=result.stdout,
                                                stderr=result.stderr)  
    
    outputs = [f for f in os.listdir(working_directory) if f.endswith('.odb')]
    print(f"Simulation finished, odb files generated.\n")
    
    return {"output_files": [os.path.join(working_directory, f) for f in outputs],
            "outputs_files_directory": [os.path.abspath(working_directory)]}


### Run Marco_node
@Workflow.wrap.as_macro_node("output_files","outputs_files_directory")
def running_exe_in_compiler(macro, compiler_path, exe_path, base_path, directory_name,
                            input_txt_paths_von_FAMOS=[], input_txt_paths_von_inp=[]):
    # path = macro.as_path() # This gives you the path based on the workflow name
    path = base_path
    macro.work_dir = generate_working_directory_overwrite(path, directory_name)
    # macro.working_directory = generate_working_directory_keep(path)
    macro.write_exe_input = write_running_exe_input(compiler_path=compiler_path, 
                                                    exe_path=exe_path, 
                                                    working_directory=macro.work_dir,
                                                    input_txt_paths_von_FAMOS=input_txt_paths_von_FAMOS,
                                                    input_txt_paths_von_inp=input_txt_paths_von_inp)
    macro.fortran_to_exe = run_exe(compiler_path=macro.write_exe_input["compiler_path"], 
                                   exe_filename=macro.write_exe_input["exe_filename"],
                                   working_directory=macro.work_dir)

    return (macro.fortran_to_exe["output_files"],
            macro.fortran_to_exe["outputs_files_directory"])

In [34]:
### Marco_node: Pre-processing, find the files number and change the name of basic file

@Workflow.wrap.as_function_node
def processing_mech_files(working_directory):
    index = 250822
    find_mech_file = f"{index}_MECH_START.odb"
    replace_mech_file = f"{index}_MECH_JOINED.odb"

    find_path = os.path.join(working_directory, find_mech_file)
    replace_path = os.path.join(working_directory, replace_mech_file)

    if os.path.exists(find_path):
        shutil.copy(find_path, replace_path)
        print(f"aimed odb file found und renamed a new file")
    else:
        print(f"aimed odb file does not exist")

    
    pattern = re.compile(rf"{index}_WPS_MECH_RESTART(\d+)\.odb$")
    max_num = -1
    for filename in os.listdir(working_directory):
        match = pattern.match(filename)
        if match:
            number = int(match.group(1))
            if number > max_num:
                max_num = number

    if max_num >= 0:
        print(f"maximal RESTART file number is: {max_num}")
    else:
        print(f"does not find RESTART file, please check the directory")

    return {"Initialization":replace_path,
            "Iteration_number":max_num}


### Run Marco_node
@Workflow.wrap.as_macro_node("Initialization","Iteration_number")
def Mechfiles_number_and_rename_Mechfile(macro, working_directory):

    macro.preprocessing_mech_files = processing_mech_files(working_directory=working_directory)

    return (macro.preprocessing_mech_files["Initialization"],
            macro.preprocessing_mech_files["Iteration_number"])

In [35]:
### Marco_node: Pre-processing, change the parameter(files number) in join.for and running in compiler, in order to get modified join.exe

@Workflow.wrap.as_function_node
def write_join_input(compiler_path, fortran_join_path, working_directory, Iteration_number):
    compiler_path = os.path.abspath(compiler_path)
    fortran_join_path = os.path.abspath(fortran_join_path)

    input_fortran_file = os.path.basename(fortran_join_path)   # Script name only (no path)
    local_fortran_join_path = os.path.join(working_directory, input_fortran_file)   # Spell out the target path in the working directory
    shutil.copy(fortran_join_path, local_fortran_join_path)

    with open(local_fortran_join_path, "r") as f:
        lines = f.readlines()

    updated_lines = []
    Iteration_number = int(Iteration_number)
    for line in lines:
        if "do i=0," in line.lower():
            parts = line.split("do i=0,")
            old_fisrt = parts[0]
            # old_second = parts[1]
            new = f"{old_fisrt}do i=0, {Iteration_number}\n"
            updated_lines.append(new)
        else:
            updated_lines.append(line)

    with open(local_fortran_join_path, "w") as f:
        f.writelines(updated_lines)
    

    return {"compiler_path":compiler_path, 
            "fortran_join_path":local_fortran_join_path}


@Workflow.wrap.as_function_node
def run_fortran_join_in_compiler(compiler_path, local_fortran_join_path, working_directory):
    ## Parsing .lnk shortcuts (Windows)
    with open(compiler_path, 'rb') as f:
        lnk = pylnk3.parse(f)

    raw_path = lnk.path.strip('"')    # Executable path to which the shortcut points
    raw_args = lnk.arguments.strip()    # Shortcut parameters

    ## If .lnk points to cmd.exe, it is necessary to extract the .bat path from the parameter
    if raw_path.lower().endswith("cmd.exe"):
        match = re.search(r'"(?P<bat>C:.*?\.bat)"\s*(?P<args>[^"]*)', raw_args, re.IGNORECASE)
        if not match:
            raise ValueError(f"Unable to extract .bat file path from .lnk arguments with contents：{raw_args}")
        env_bat_path = match.group("bat")
        extra_args = match.group("args")
    else:
        env_bat_path = raw_path
        extra_args = raw_args

    ## Constructing the ifort compilation command
    # exe_name = self.output_exe or os.path.splitext(os.path.basename(self.local_fortran))[0] + ".exe"
    # exe_name = os.path.normpath(exe_name)
    fortran_path = os.path.normpath(local_fortran_join_path)
    compile_cmd = f'ifort "{fortran_path}"'

    setup_cmd = f'call "{env_bat_path}" {extra_args} && {compile_cmd}'

    # print(setup_cmd)

    try:
        result = subprocess.run(
            f'cmd /c "{setup_cmd}"',
            cwd=working_directory,
            shell=True,
            check=True,
            text=True,
            capture_output=True
        )
        print("compiler output:\n", result.stdout)
        if result.stderr:
            print("warning:\n", result.stderr)
    except subprocess.CalledProcessError as e:
        print("error")
        print("stdout:\n", e.stdout)
        print("stderr:\n", e.stderr)
        raise

    exe_files = [f for f in os.listdir(working_directory) if f.lower().endswith(".exe")]
    if not exe_files:
        raise FileNotFoundError(".exe file not found in the working directory")

    exe_filename = exe_files[0]
    exe_path = os.path.abspath(os.path.join(working_directory, exe_filename))
    
    print(f"Finish, generated file: ['{exe_filename}']\n")
    
    return {"fortran_exe_filename": exe_filename,
            "exe_path": exe_path}


### Run Marco_node
@Workflow.wrap.as_macro_node("fortran_join_exe_filename", "join_exe_path")
def fortran_join_to_exe(macro, compiler_path, fortran_join_path, base_path, directory_name, Iteration_number):
    # path = macro.as_path() # This gives you the path based on the workflow name
    path = base_path
    macro.work_dir = generate_working_directory_overwrite(path, directory_name)
    # macro.working_directory = generate_working_directory_keep(path)
    macro.fortran_join_file = write_join_input(compiler_path=compiler_path, 
                                               fortran_join_path=fortran_join_path, 
                                               working_directory=macro.work_dir,
                                               Iteration_number=Iteration_number)
    macro.fortran_join_to_exefile = run_fortran_join_in_compiler(compiler_path=macro.fortran_join_file["compiler_path"], 
                                                             local_fortran_join_path=macro.fortran_join_file["fortran_join_path"],
                                                             working_directory=macro.work_dir)

    return (macro.fortran_join_to_exefile["fortran_join_exe_filename"],
            macro.fortran_join_to_exefile["join_exe_path"])

In [36]:
### Marco_node: join all .odb files from simulation

@Workflow.wrap.as_function_node
def write_join_exe_input(compiler_path, exe_path, Initial_file_path, working_directory, input_odb_paths):
    compiler_path = os.path.abspath(compiler_path)
    exe_path = os.path.abspath(exe_path)
    Initial_file_path = os.path.abspath(Initial_file_path)
    
    # Copy .exe to working directory
    exe_filename = os.path.basename(exe_path)   # Script name only (no path)
    local_exe_path = os.path.join(working_directory, exe_filename)   # Spell out the target path in the working directory
    shutil.copy(exe_path, local_exe_path)

    # Copy initial.odb to working directory
    initial_odb_filename = os.path.basename(Initial_file_path)   # Script name only (no path)
    local_initial_odb_path = os.path.join(working_directory, initial_odb_filename)   # Spell out the target path in the working directory
    shutil.copy(Initial_file_path, local_initial_odb_path)
    
    # Copy all input odb files into working directory
    local_odb_files = []
    for odb_path in input_odb_paths:
        odb_name = os.path.basename(odb_path)
        dst_path = os.path.join(working_directory, odb_name)
        odb_path = os.path.abspath(txt_path)
        shutil.copy(odb_path, dst_path)
        local_odb_files.append(dst_path)
        
    # print(local_txt_files)
    
    return {"compiler_path": compiler_path,
            "join_exe_filename": join_exe_filename,
            "input_odb_set": local_odb_files}


@Workflow.wrap.as_function_node
def run_join_exe(compiler_path, exe_filename, working_directory):
    with open(compiler_path, 'rb') as f:
        lnk = pylnk3.parse(f)

    raw_path = lnk.path.strip('"')
    raw_args = lnk.arguments.strip()

    if raw_path.lower().endswith("cmd.exe"):
        match = re.search(r'"(?P<bat>C:.*?\.bat)"\s*(?P<args>[^"]*)', raw_args, re.IGNORECASE)
        if not match:
            raise ValueError(f"Unable to extract .bat file path from .lnk arguments with contents：{raw_args}")
        env_bat_path = match.group("bat")
        extra_args = match.group("args").strip('"')
    else:
        env_bat_path = raw_path
        extra_args = raw_args
        
    ## First call the compiler environment variable
    ## Then cd to the working directory
    ## Finally, execute the .exe file
    run_cmd = f'call "{env_bat_path}" {extra_args} && cd /d "{working_directory}" && "{exe_filename}"'

    # print(f"\n{run_cmd}\n")

    try:
        result = subprocess.run(
            f'cmd /c "{run_cmd}"',
            cwd=working_directory,
            shell=True,
            check=True,
            capture_output=True,
            text=True
        )
        print(result.stdout)
        print(result.stderr)
        
    except subprocess.CalledProcessError as e:
        print("error：")
        print("STDOUT:\n", e.stdout)
        print("STDERR:\n", e.stderr)
        raise

    outputs = [f for f in os.listdir(working_directory) if f.endswith('.odb') and f == initial_odb_filename]
    
    print(f"Joining finished, final odb files generated.\n")
    
    return {"Final_odb_file_path": [os.path.join(working_directory, f) for f in outputs]}


### Run Marco_node
@Workflow.wrap.as_macro_node("Final_odb_file_path")
def running_join_exe_in_compiler(macro, compiler_path, exe_path, base_path, 
                                 Initial_file_path, directory_name, input_odb_paths):
    # path = macro.as_path() # This gives you the path based on the workflow name
    path = base_path
    macro.work_dir = generate_working_directory_overwrite(path, directory_name)
    # macro.working_directory = generate_working_directory_keep(path)
    macro.write_join_exe_input = write_join_exe_input(compiler_path=compiler_path, 
                                                      exe_path=exe_path, 
                                                      Initial_file_path=Initial_file_path,
                                                      working_directory=macro.work_dir,
                                                      input_odb_paths=input_odb_paths)
    macro.fortran_join_to_exe = run_join_exe(compiler_path=macro.write_join_exe_input["compiler_path"], 
                                             exe_filename=macro.write_join_exe_input["join_exe_filename"],
                                             working_directory=macro.work_dir)

    return (macro.fortran_join_to_exe["Final_odb_file_path"])

In [37]:
if __name__ == '__main__':

    wf = Workflow("Simulation_Workflow")
    wf.children.clear()
    
    project_path = r"C:\Local_Dong\Projekt\AnAttAl\CAD\AnAttAl_CAD_OnlyWorkflow_demo"
    abaqus_python_script = r"C:\Local_Dong\Projekt\AnAttAl\CAD\abaqusMacros.py"
    processing_inp_python_script = r"C:\Local_Dong\Projekt\AnAttAl\CAD\File_Programm2.py"
    fortran_compiler_path = r"C:\Local_Dong\Projekt\AnAttAl\CAD\Compiler 16.0 Update 1 for Intel 64 Visual Studio 2015 environment.lnk"
    # fortran_file = r"C:\Local_Dong\Projekt\AnAttAl\CAD\Testfall_1ms_Auskommentiert.for" 
    fortran_file = r"C:\Local_Dong\Projekt\AnAttAl\CAD\Testfall_1ms_Dyn5_a_a4.for" 
    fortran_join_file = r"C:\Local_Dong\Projekt\AnAttAl\CAD\odbjoin.for"
    input_txts_FAMOS = [r"C:\Local_Dong\Projekt\AnAttAl\CAD\Stromverlauf.txt",
                        r"C:\Local_Dong\Projekt\AnAttAl\CAD\Kraftverlauf.txt",
                        r"C:\Local_Dong\Projekt\AnAttAl\CAD\Potentialverlauf.txt"]
    
    wf.Update_Geo_Abaqus = update_geometry_marco(original_abaqus_script=abaqus_python_script,
                                                 working_directory=project_path)
    
    wf.Abaqus_Output = export_inp_cae(script_path=wf.Update_Geo_Abaqus.outputs["new_py_path"], 
                                      base_path=project_path,
                                      directory_name="Python_marco_to_inp")
    
    wf.Processing_inp = processing_inp(script_path=processing_inp_python_script, 
                                       inp_path=wf.Abaqus_Output.outputs["inp_path"], 
                                       base_path=project_path,
                                       directory_name="Python_marco_processing_inp")

    wf.FAMOS_Output = FAMOS_Output(base_path=project_path,
                                   directory_name="FAMOS_Output",
                                   input_txt_paths_von_FAMOS=input_txts_FAMOS)
    
    wf.Fortran_Executable_File = fortran_to_exe(compiler_path=fortran_compiler_path, 
                                                fortran_file_path=fortran_file, 
                                                base_path=project_path, 
                                                directory_name="Fortran_to_simulation_exe")
    
    wf.Executing_File_in_compiler = running_exe_in_compiler(compiler_path=fortran_compiler_path,
                                                            exe_path=wf.Fortran_Executable_File.outputs["exe_path"],
                                                            base_path=project_path,
                                                            directory_name="running_exe_in_compiler",
                                                            input_txt_paths_von_FAMOS=wf.FAMOS_Output.outputs["FAMOS_txt"],
                                                            input_txt_paths_von_inp=wf.Processing_inp.outputs["txt_paths"])

    wf.Number_of_Iterations_and_Initialization = Mechfiles_number_and_rename_Mechfile(working_directory=wf.Executing_File_in_compiler.outputs["outputs_files_directory"])

    wf.Fortran_join_Executable_File = fortran_join_to_exe(compiler_path=fortran_compiler_path,
                                                          fortran_join_path=fortran_join_file,
                                                          base_path=project_path,
                                                          directory_name="Fortran_to_join_exe",
                                                          Iteration_number=wf.Number_of_Iterations_and_Initialization.outputs["Iteration_number"])

    wf.Executing_join_in_compiler = running_join_exe_in_compiler(compiler_path=fortran_compiler_path,
                                                                 input_odb_paths=wf.Executing_File_in_compiler.outputs["output_files"],
                                                                 base_path=project_path,
                                                                 directory_name="executing_join_exe",
                                                                 exe_path=wf.Fortran_join_Executable_File.outputs["join_exe_path"],
                                                                 Initial_file_path=wf.Number_of_Iterations_and_Initialization.outputs["Initialization"])

    wf.draw(size=(10,10))
    # wf.run()


In [28]:
    pf = PyironFlow([wf])
    pf.gui